### Digital Cut Up Revisit

In my first cut up, I made a list of phrases/fragments of text from Clarice Lispector's "The Hour of the Star". I chose this book because I was interested in the way the narration is intrusive and called me out. 

In the introduction to the book Colm Tóibín describes: 

*"As the French critic Hélène Cixous has written, The Hour of the Star is mysterious, garrulous and oddly refined. It withholds and it tells too much. It makes sweeping judgments and tiny observations. It is a meditation on two types of powerlessness, each one stark and distinct. The first is the powerlessness of the narrator, someone who has words at his disposal but who feels that words, in all their uncertainty and shiftiness, will dispose of him. He is not sure whether this should make him laugh or cry; instead he remains in an odd, frightened state with strange bursts of pure determination. And then there is the powerlessness of the character he has imagined, or seen, or allowed the words, in all their frailty and foolishness, to conjure up."*

This time I used spacy to isolate parts of speech and followed the tracery example. 

I also found out Lispector included thirteen alternative titles: 
* "The Hour of the Star", 
* "It's All My Fault", 
* "Let Her Deal With It", 
* "The Right to Scream", 
* ".As for the Future.", 
* "Singing the Blues", 
* "She Doesn't Know How to Scream", 
* "A Sense of Loss", 
* "Whistling in the Dark Wind", 
* "I Can't Do Anything", 
* "Account of the Preceding Facts", 
* "Cheap Tearjerker", 
* "Discreet Exit Through the Back Door".

In [66]:
import spacy
nlp = spacy.load('en_core_web_md')

In [68]:
text = open("2try.txt").read()
doc = nlp(text)

In [69]:
sentences = list(doc.sents)
words = [w for w in list(doc) if w.is_alpha]
noun_chunks = list(doc.noun_chunks)
entities = list(doc.ents)
times = [e for e in entities if e.label_ == "TIME"]

In [84]:
import random
for item in random.sample(noun_chunks, 5):
    print(item.text.strip().replace("\n", " "))
    print()

it

a kind

I

it

it



In [88]:
for item in random.sample(locations, 18): # change "times" to "people" or "locations" to sample those lists
    print(item.text.strip())

NameError: name 'locations' is not defined

In [89]:
def flatten_subtree(st):
    return ''.join([w.text_with_ws for w in list(st)]).strip()

In [90]:
prep_phrases = []
for word in doc:
    if word.dep_ == 'prep':
        prep_phrases.append(flatten_subtree(word.subtree).replace("\n", " "))

In [92]:
random.sample(prep_phrases, 100)

['of courtly love',
 'in my arms',
 'of images',
 'on his engagement calendar,',
 'of the other',
 'upon its affirmation',
 'for your foot (your size and your pleasure',
 'of necessity',
 'except for one',
 'in the dinner',
 'As a child',
 'in its way',
 'to that black economy which marks me with its aberration',
 'around me',
 'In the text',
 'in such a way',
 'in the image',
 'in the other',
 'to him',
 'not in its rhetorical sense, but rather in its gymnastic or choreographic acceptation',
 "into the grammar of the lover's discourse",
 'of which',
 'into mourning for his patient (or else',
 'than H." Inexorable',
 'than the flight of mosquitoes',
 'of pathos',
 'of my maxims',
 'on the telephone',
 'to the spot',
 'without limit',
 'in despair or fulfillment',
 'in the subject',
 'of the Shades',
 'in Sade',
 'of modern souls',
 "as for Bataille's subject to take off his clothes in a public place",
 'of',
 'into this prospect',
 'of the alternative',
 'with a rival person',
 'though

In [93]:
subjects = [flatten_subtree(word.subtree).replace("\n", " ")
            for word in doc if word.dep_ in ('nsubj', 'nsubjpass')]
past_tense_verbs = [word.text for word in words if word.tag_ == 'VBD' and word.lemma_ != 'be']
adjectives = [word.text for word in words if word.tag_.startswith('JJ')]
nouns = [word.text for word in words if word.tag_.startswith('NN')]
prep_phrases = [flatten_subtree(word.subtree).replace("\n", " ")
                for word in doc if word.dep_ == 'prep']

In [94]:
import tracery
import textwrap
from tracery.modifiers import base_english

In [107]:
rules = {
    "origin": [
        "#prepphrase.capitalize#, you #predicate#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#noun-chunks.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "verb": past_tense_verbs,
    "noun": nouns,
    "noun-chunks": noun_chunks,
    "adj": adjectives,
    "prepphrase": prep_phrases
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'Of proffering the I-love-you which I extend:, you had of course.'

In [108]:
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(12)])
#print(fill(output, 60))
print(textwrap.fill(output, 60))

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.token.Token, got str)

In [109]:
# she mentioned a book she had just completed with "thirteen names, thirteen titles", referencing the thirteen alternative titles on the title page of the novella
thirteen = [
    "The Hour of the Star", "It's All My Fault", "Let Her Deal With It", "The Right to Scream", ".As for the Future.", "Singing the Blues", "She Doesn't Know How to Scream", "A Sense of Loss", "Whistling in the Dark Wind", "I Can't Do Anything", "Account of the Preceding Facts", "Cheap Tearjerker", "Discreet Exit Through the Back Door"
    ]

In [114]:
rules = {
    "origin": [ 
        "#prepphrase.capitalize#, you #predicate#.",
        "You #predicate# #prepphrase#."
    ],
    "predicate": [
        "#verb#",
        "#verb# #nounphrase#",
        "#verb# #prepphrase#"
    ],
    "nounphrase": [
        "the #noun#",
        "the #adj# #noun#",
        "the #noun# #prepphrase#",
        "the #noun# and the #noun#",
        "#noun.a#",
        "#noun-chunks.a#",
        "#adj.a# #noun#",
        "the #noun# that #predicate#"
    ],
    "verb": past_tense_verbs,
    "noun": nouns,
    "noun-chunks": noun_chunks,
    "adj": adjectives,
    "prepphrase": prep_phrases,
    "titles": thirteen
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

'Of morality, you struggled.'

## 13 Stories for Lispector's Alternative Names

In [283]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(13)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

.As for the Future.
For myself, you lived for a very brief interval I surprise in the other, detached from the rest of
his person. In Praise of, you cluded. As a tiny figure, you had a splendid delays. You made of my
speech. Into other words" the ipseity of my Image,, you loved of Folly. Without seeming to, looking
for who to love, you dreaded. You had in love. You occurred a Parsifal like a bureaucracy. Of, you
said an other friend. Of rapid algebra which my discourse requires at this particular moment, you
lyzed. You regretted a sound in the "nature" of amorous madness to pass,. You had of non-mastery. Of
my habits, you loved.


In [298]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(12)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

She Doesn't Know How to Scream
You vanished with. Of a fetish, you rested of wearing it. You asked of nothing from anything
inexpressible. As an amorous sub­ ject, you loved. Of a discrete pathos, you saw the effect. In
short, you saw the body to. You asked a higher lover in one way or another. You cluded from it on
the floor of the sea. You blamed for myself onto others. In the face, you asked though at each slice
which she offered, for politeness's sake,. You conspired as subject. You remained by an oral
pleasure underneath the lamp.


In [322]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(11)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

The Hour of the Star
On the same level, you saw the idea that vanished. You replied From any consultant whatever in order
to preserve the very space of dependency,. You waken of-pearl. From Phalerum,, you suspended at
last. You had a common love from desire. You became a society for. On the one hand, you had in love.
In a second, you saw. You finished a nalized body in her. Like Pelleas emerging from the underground
chambers and rediscovering life,, you struggled. You jected an other Expenditure than a dust of
figures stirring according to an unpredictable order.


In [152]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(10)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.token.Token, got str)

In [248]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(9)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

Singing the Blues
You fell of the Cross                    Ruysbroeck from it. To the category of the bores, you
weighed. Than an impulse, you uttered in a public place. You seized of their misfortune. Of Holland,
you begged the bad Amadour. Only by looks, expressions, faces), you drove of roles. In short,, you
said the Way. You explained the other reading of a general situation. Than the flight of mosquitoes,
you dared To the beloved.


In [252]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(8)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

It's All My Fault
You created the subject that remplit to my misfortune of a fulfilled future. To (I obtain?), you did
of two peacocks. Of language which oppresses and repulses me, you explained a wall. You chased not
as a fact but in the aspect of a sign. As a writer, you guessed particularly of writing. You loved
the method that departed to this ques­ tion to the world. On the other, you rowed the body from his
waiting. From this principle, you wanted the butit that had the ingenuity in   short.


In [253]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(7)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

Account of the Preceding Facts
You shut a partner of languorous desire, of the image-repertoire, of declarations. You received in
such discussion without cause and without conclusion. You depended a position of the social world-
common­ place. Of me, you died. After a long period of investiga­ tion he manages that to dissociate
the two inferences, you checked. Like the pregnant moment (pregnant= endowed with a strong, chosen
meaning) of bourgeois drama, you regretted the reception in the face. You drained not in its
rhetorical sense, but rather in its gymnastic or choreographic acceptation.


In [260]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(6)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

The Right to Scream
Between these objects and your person, you closed a desire. You made of them without warning. You
made the avowal against "my truth" of incidents. You came to a type. By himself, you reversed. You
thought of these points without pain.


In [262]:
import textwrap
from textwrap import fill
output = "\n".join([grammar.flatten("#origin#") for i in range(5)])
print(random.choice(thirteen))
print(textwrap.fill(output, 60))

The Right to Scream
You united a non place from his name. Of inexplicable
reasons and impaired solutions, you knew of memory. Without
pres­, you visited. On my side, you pigeonholed in the void.
You let the solitude and the thing of the amorous body.


In [264]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(4)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

.As for the Future.
You danced in himself toward him. To whom, you shut of what I love according to truth. From all
gregarity, you ate the days that pigeonholed. In the rain, you shut of all.


In [268]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(3)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

I Can't Do Anything
You did of which at me. You ran the little images on the scene. You loved at night.


In [26]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(2)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

Account of the Preceding Facts
To possible imaginings of yours, perhaps even depraved and pitiless, you meant a Nobre. By a very
slight and constant toothache, you did with a trembling hand.


In [144]:
import textwrap
from textwrap import fill
output = " ".join([grammar.flatten("#origin#") for i in range(1)])
print(random.choice(thirteen))
print(textwrap.fill(''.join(output), 100))

The Hour of the Star
You created as in the fairy tales to what has just been said.
